# 24. 类代码编写细节（Class Coding Details）

类的细节决定可维护性：__repr__/__str__ 便于调试；dataclasses 减少样板；默认值与可变字段要谨慎。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 22 节：类基础


## 知识点地图

- 1. __repr__ vs __str__：调试输出 vs 用户友好输出
- 2. dataclass：自动生成 __init__/__repr__/__eq__
- 3. 默认值与 default_factory：避免共享可变默认值
- 4. frozen/order（了解）：不可变与排序支持


## 自检清单（学完打勾）

- [ ] 理解 __repr__ 与 __str__ 的区别
- [ ] 会用 @dataclass 简化类定义
- [ ] 理解 dataclass 默认值与 default_factory
- [ ] 了解 frozen/order 等常用参数


## 知识点 1：__repr__ vs __str__：调试输出 vs 用户友好输出

print(obj) 走 __str__；容器打印/调试更依赖 __repr__。


In [ ]:
class User:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f'User(name={self.name!r})'

    def __str__(self):
        return self.name

u = User('Ada')
print(u)
print([u])


## 知识点 2：dataclass：自动生成 __init__/__repr__/__eq__

dataclass 适合“数据载体”类；可减少大量样板代码。


In [ ]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

p = Point(1, 2)
print(p)


## 知识点 3：默认值与 default_factory：避免共享可变默认值

可变字段默认值用 default_factory，避免多个实例共享同一个 list/dict。


In [ ]:
from dataclasses import dataclass, field

@dataclass
class Bag:
    items: list = field(default_factory=list)

b1 = Bag(); b2 = Bag()
b1.items.append('x')
print(b1.items, b2.items)


## 知识点 4：frozen/order（了解）：不可变与排序支持

frozen=True 让实例更像不可变对象；order=True 自动生成比较方法。


In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Task:
    priority: int
    title: str

print(sorted([Task(2, 'b'), Task(1, 'a')]))


## 常见坑

- dataclass 里可变默认值要用 default_factory
- __repr__ 过长会影响调试体验（保留关键字段即可）


## 综合小案例：实现一个可调试的配置类（dataclass）

写 Config(host, port, tags=list)，tags 用 default_factory；并实现 validate 方法做校验。


In [ ]:
from dataclasses import dataclass, field

@dataclass
class Config:
    host: str
    port: int
    tags: list = field(default_factory=list)

    def validate(self):
        if not self.host:
            raise ValueError('host required')
        if not (0 < self.port < 65536):
            raise ValueError('invalid port')

cfg = Config('127.0.0.1', 8080, tags=['dev'])
cfg.validate()
print(cfg)


## 自测题（不写代码也能回答）

- __repr__ 与 __str__ 分别用于什么场景？
- dataclass 为什么适合做数据类？
- default_factory 解决了什么问题？


## 练习题（建议写代码）

- 把你第 23 节写的某个类改成 dataclass（如果合适）。
- 写一个 frozen 的 Money 类（amount, currency），并验证无法修改字段。
